# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

df = pd.read_csv("homepage_actions.csv")

In [74]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [75]:
counts = df.id.value_counts()

twos = counts[counts.values == 2]
# checks for any double views or double clicks
for id in twos.keys():
    doubles = []
    if sum(df.action[df.id == id].map({"view":1, "click":0})) != 1:
        doubles.append(id)
print("Number of double views or double clicks: %s" % (len(doubles)))


num_overlapping = sum((df[df.id.isin(twos.keys())].groupby("id")).group.nunique().eq(1) != 1)
print("Number of overlapping groups: %s" % (num_overlapping))


# checks for clicks that didnt view
ones = counts[counts.values == 1]
only_click = sum(df.action[df.id.isin(ones.keys())].values != "view")
print("Number that clicked that didnt view: %s" % (only_click))




Number of double views or double clicks: 0
Number of overlapping groups: 0
Number that clicked that didnt view: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [111]:
rels = df.sort_values("timestamp").drop_duplicates(subset=["id"], keep="last")
counts = rels.groupby(["group", "action"]).count().id

control = counts.values[:2]
control = control / control.sum()
exp = counts.values[-2:]
exp = exp / exp.sum()

print(control, exp)
stats.chisquare(control, exp)


[0.27971188 0.72028812] [0.30974633 0.69025367]


Power_divergenceResult(statistic=0.004219143450475796, pvalue=0.9482098818973134)

# Results
With a p-value of 0.97 (> alpha of 0.05), we can conclude there was **no statistical difference** between
conversion rates of the experimental vs control homepage

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [128]:
ctrl_counts = df[df.group == "control"].action.value_counts()
control_ctr = ctrl_counts["click"] / ctrl_counts["view"]

exp_counts = df[df.group == "experiment"].action.value_counts()
exp_counts
exp_clicks = int(exp_counts["view"] * control_ctr)
exp_clicks


838

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [129]:
rels["binary"] = rels.action.map({"view":0, "click":1})

rels.binary.std()

0.45559704843233867

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
stats.ttest_ind()

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.